<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-正则线性模型" data-toc-modified-id="1.-正则线性模型-1">1. 正则线性模型</a></span><ul class="toc-item"><li><span><a href="#4.1-套索回归（L1正则化）" data-toc-modified-id="4.1-套索回归（L1正则化）-1.1">4.1 套索回归（L1正则化）</a></span></li><li><span><a href="#4.2-岭回归（L2正则化）" data-toc-modified-id="4.2-岭回归（L2正则化）-1.2">4.2 岭回归（L2正则化）</a></span></li><li><span><a href="#4.3-弹性网络回归（L1&amp;L2）" data-toc-modified-id="4.3-弹性网络回归（L1&amp;L2）-1.3">4.3 弹性网络回归（L1&amp;L2）</a></span></li></ul></li></ul></div>

# 1. 正则线性模型

参考：机器学习实战p121-126

**Lamda Parameter**:
* Small: Choose Complex Model
* Large: Choose Simpler Moder
![d971145b418eacaf61ea42b5f9705a3f](线性回归 - Linear Regression.resources/062F8E59-828C-4E6D-8C76-5D1E52A938B5.png)
![c1e465e11ce4f6f36c35e316073934ce](线性回归 - Linear Regression.resources/E9B8DABF-A90B-40AF-A1A4-093B24A05292.png)

## 4.1 套索回归（L1正则化）

Lasso Regression

* 适用于稀疏数据（如1000列数据中只有10列相关）
![e11404ca400bd5380f190f4643819893](线性回归 - Linear Regression.resources/740850D9-B056-449D-AB6D-0150FAE76561.png)

## 4.2 岭回归（L2正则化）

Ridge Regression

* 适用于非稀疏数据
![201db7d35ce6ac39c6d62ed465bd3ba8](线性回归 - Linear Regression.resources/17C773A0-5F68-4E37-8A56-52D57B88926A.png)

* * *

## 4.3 弹性网络回归（L1&L2）

Elastic Net